In [2]:
import os
import numpy as np
import pandas as pd
from datetime import date

from sklearn.model_selection import KFold, train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import log_loss, roc_auc_score, auc, roc_curve

data_root = "../data/ML100midterm/"

In [3]:
#讀取train & test資料
dfoff = pd.read_csv(os.path.join(data_root, "train_offline.csv"))
dftest = pd.read_csv(os.path.join(data_root, "test_offline.csv"))

#刪減資料並重新排列
dftest = dftest[~dftest.Coupon_id.isna()]
print(dfoff.shape)
print(dftest.shape)
dfoff.head(20)

(1160742, 7)
(306313, 6)


,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN
5,2223968,3381,9776.0,10:5,2.0,20160129.0,NaN
6,73611,2099,12034.0,100:10,NaN,20160207.0,NaN
7,163606,1569,5054.0,200:30,10.0,20160421.0,NaN
8,3273056,4833,7802.0,200:20,10.0,20160130.0,NaN
9,94107,3381,7610.0,200:20,2.0,20160412.0,NaN


In [4]:
#建立label（0,1），並將train的資料定義label，作為最後判定是否使用消費卷的指標（預測結果）
"""""
1) buy with coupon within (include) 15 days ==> 1
2) buy with coupon but out of 15 days ==> 0
3) buy without coupon ==> -1 (we don't care)
"""""

def label(row):
    if np.isnan(row["Date_received"]):
        return -1
    if not np.isnan(row["Date"]):
        td = pd.to_datetime(row["Date"], format="%Y%m%d") - pd.to_datetime(row["Date_received"], format="%Y%m%d")
        if td <= pd.Timedelta(15, "D"):
            return 1
    return 0    

dfoff["label"] = dfoff.apply(label, axis=1)
dfoff["label"].value_counts()

 0    710665
-1    413773
 1     36304
Name: label, dtype: int64

In [5]:
#建立特徵weekday(星期幾)
def getWeekday(row):
    if (np.isnan(row)) or (row==-1):
        return row
    else :
        return pd.to_datetime(row, format="%Y%m%d").dayofweek+1

dfoff["weekday"] = dfoff["Date_received"].apply(getWeekday)
dftest["weekday"] = dftest["Date_received"].apply(getWeekday)


#建立特徵weekday_type(是否為週末1,0)
dfoff["weekday_type"] = dfoff["weekday"].astype("str").apply(lambda x: 1 if x in[6,7] else 0)
dftest["weekday_type"] = dftest["weekday"].astype("str").apply(lambda x: 1 if x in[6,7] else 0)

In [6]:
#建立特徵weekday_cols，標籤編碼並建立pandas表格=>建立column=>將pandas值與column match(星期一～日皆是獨立特徵)
weekdaycols = ["weekday_" + str(i) for i in range(1,8)]
print(weekdaycols)

tmpdf = pd.get_dummies(dfoff["weekday"].replace(-1, np.nan))
tmpdf.columns = weekdaycols
dfoff[weekdaycols] = tmpdf

tmpdf = pd.get_dummies(dftest["weekday"].replace(-1, np.nan))
tmpdf.columns = weekdaycols
dftest[weekdaycols] = tmpdf

['weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7']


In [7]:
#建立特徵getDiscountType、convertRate、getDiscountMan、getDiscountJian （利用Discount_rate、Distance轉換）
def getDiscountType(row):
    if row == "null":
        return "null"
    elif ":" in row:
        return 1
    else:
        return 0

    
def convertRate(row):
    if row == "null":
        return 1.0
    elif ":" in row:
        rows = row.split(":")
        return 1.0 - float(rows[1]) / float(rows[0])
    else:
        return float(row)
    

def getDiscountMan(row):
    if ":" in row:
        rows = row.split(":")
        return int(rows[0])
    else:
        return 0

def getDiscountJian(row):
    if ":" in row:
        rows = row.split(":")
        return int(rows[1])
    else:
        return 0

    
def processData(df):
    df["discount_type"] = df["Discount_rate"].astype("str").apply(getDiscountType)
    df["discount_rate"] = df["Discount_rate"].astype("str").apply(convertRate)
    df["discount_man"] = df["Discount_rate"].astype("str").apply(getDiscountMan)
    df["discount_jian"] = df["Discount_rate"].astype("str").apply(getDiscountJian)
    
    df.loc[df.Distance.isna(), "Distance"] = 99
    return df
    

dfoff = processData(dfoff)
dftest = processData(dftest)

In [8]:
#將訓練資料(train data)分成兩組train & valid
def split_train_valid(row, date_cut = "20160416"):
    is_train = True if pd.to_datetime(row, format="%Y%m%d") < pd.to_datetime(date_cut, format="%Y%m%d") else False
    return is_train

df = dfoff[dfoff["label"]!=-1].copy()
df["is_train"] = df["Date_received"].apply(split_train_valid)

train = df[df["is_train"]]
valid = df[~df["is_train"]]

train.reset_index(drop=True, inplace=True)
valid.reset_index(drop=True, inplace=True)

print("train size:{}, #positive:{}".format(len(train), train["label"].sum()))
print("valid size:{}, #positive:{}".format(len(valid), valid["label"].sum()))

train size:667753, #positive:32472
valid size:79216, #positive:3832


In [9]:
#列出主要訓練資料使用的特徵值
original_feature = ["discount_type", "discount_rate", "discount_man", "discount_jian", "Distance", "weekday", "weekday_type"] + weekdaycols
print(len(original_feature), original_feature)

14 ['discount_type', 'discount_rate', 'discount_man', 'discount_jian', 'Distance', 'weekday', 'weekday_type', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7']


In [10]:
#建立model，先利用StandardScaler() ＆ classifier()，並利用GridSearchCV取得最佳參數
predictors = original_feature
print(predictors)

def check_model(data, predictors):
    classifier = lambda: SGDClassifier(loss="log", penalty="elasticnet", shuffle=True, fit_intercept=True, max_iter=100, n_jobs=1, class_weight=None)
    
    model = Pipeline(steps=[("ss", StandardScaler()), ("en", classifier())])
    
    parameters = {"en__alpha":[0.001, 0.01, 0.1], "en__l1_ratio":[0.001, 0.01, 0.1]}
                    
    folder = StratifiedKFold(n_splits=3, shuffle=True)
                     
    
    grid_search = GridSearchCV(model, parameters, cv=folder, n_jobs=-1, verbose=1)
    grid_search = grid_search.fit(data[predictors], data["label"])
    
    return grid_search

['discount_type', 'discount_rate', 'discount_man', 'discount_jian', 'Distance', 'weekday', 'weekday_type', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7']


In [11]:
#將train data帶入model去fit
model = check_model(train, predictors)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:  1.9min finished
/Users/daihongming/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/daihongming/anaconda3/lib/python3.7/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/Users/daihongming/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21

In [12]:
#用valid data去預測結果，predict_proba結果為兩行(一行為預測標籤為0的機率，一行為預測標籤為1的機率)
y_valid_pred = model.predict_proba(valid[predictors])
valid1 = valid.copy()
valid1["valid_prob"] = y_valid_pred[:, 1]

/Users/daihongming/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:381: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [20]:
y_valid_pred[:, 1]

array([0.00686939, 0.00601692, 0.00926648, ..., 0.00807127, 0.00681002,
       0.00546361])

In [21]:
#計算valid data預測準度(利用auc_roc_score, accuracy_socre)
from sklearn.metrics import roc_auc_score, accuracy_score
auc_score = roc_auc_score(y_true=valid.label, y_score=y_valid_pred[:, 1] )
acc = accuracy_score(y_true=valid.label, y_pred = y_valid_pred.argmax(axis=1))
print("Validation AUC: {:.3f}, Accuracy: {:.3f}".format(auc_score, acc))

Validation AUC: 0.743, Accuracy: 0.952


In [22]:
#測試集最後的columns
dftest.columns

Index(['User_id', 'Merchant_id', 'Coupon_id', 'Discount_rate', 'Distance',
       'Date_received', 'weekday', 'weekday_type', 'weekday_1', 'weekday_2',
       'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7',
       'discount_type', 'discount_rate', 'discount_man', 'discount_jian'],
      dtype='object')

In [23]:
#去除test data內Coupon_id為0的資料
targetset = dftest.copy()
print(targetset.shape)

targetset = targetset[~targetset.Coupon_id.isna()]
targetset.reset_index(drop=True, inplace=True)

#選出用來預測的特徵值資料(predictors)
testset = targetset[predictors].copy()

#預測test data的結果
y_test_pred = model.predict_proba(testset[predictors])
test1 = testset.copy()
test1["pred_prob"] = y_test_pred[:, 1]
print(test1.shape)

(306313, 19)
(306313, 15)


/Users/daihongming/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:381: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [48]:
#將三種id取出，並加入預測結果併成一datafram
output = pd.concat((targetset[["User_id", "Coupon_id", "Date_received"]], test1["pred_prob"]), axis=1)

output.loc[:, "User_id"] = output["User_id"].apply(lambda x: str(int(x)))
output.loc[:, "Coupon_id"] = output["Coupon_id"].apply(lambda x: str(int(x)))
output.loc[:, "Date_received"] = output["Date_received"].apply(lambda x: str(int(x)))

#將三種id整合為一column
output["uid"] = output[["User_id", "Coupon_id", "Date_received"]].apply(lambda x: "_".join(x.values), axis=1)

output.reset_index(drop=True, inplace=True)
output.head()

,User_id,Coupon_id,Date_received,pred_prob,uid
0,1439408,11002,20160528,0.013494,1439408_11002_20160528
1,1439408,8591,20160613,0.065329,1439408_8591_20160613
2,1439408,8591,20160516,0.065329,1439408_8591_20160516
3,2029232,1532,20160530,0.060906,2029232_1532_20160530
4,2029232,12737,20160519,0.098407,2029232_12737_20160519


In [54]:
#將預測結果轉換為csv檔
out = output.groupby("uid", as_index=False).mean()
out[["uid", "pred_prob" ]]
out.columns = ["uid", "label"]

out.to_csv("2nd_ML_midterm.csv", header=["uid", "label"], index=False)
out.head()

,uid,label
0,1000020_2705_20160519,0.114257
1,1000020_8192_20160513,0.088141
2,1000065_1455_20160527,0.067587
3,1000085_8067_20160513,0.070539
4,1000086_2418_20160613,0.060906
